Import modules for this pset: 

In [2]:
%load_ext autoreload
%autoreload 2
import math
import numpy

import geo_utils
import execute
import visualization

from tests import *
from shapely.geometry import Point

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'shapely'

# Semantic Localization Mini-Pset

In this pset you'll be filling in some of the basic sampling-based localization methods, following along with the creation of the environment and algorithm, and then testing your localization in a variety of ways. 

0. [About running this problem set](#about)
1. [Localization](#localization)
2. [Map-making](#map-making)
3. [Test your localization algorithm](#test-methods)
4. [What is semantic about all this?](#semantic-ness)

## About running this problem set <a id="about"/>

This problem set will require running simulations that will pop up in a new Python window. These may be slow, with each frame update taking between .5 - 3 seconds depending on how fast your computer is. The test will evaluate after you exit out of the pop-up window. <b> The window may not disappear immediately, but this does not mean the code is frozen. </b> If you force-quit the Python window, you will reset the kernel and the test will never evaluate as correct.

## Localization <a id="localization"/>

Monte-Carlo Localization is a type of sampling technique. The robot converges upon an approximate distribution of its most likely position by iterating between its current observations and sampling from points of more and more likely observations. 

Below implement a particle filter update step. 

<b>The following variables and functions are already availible to you in the `Localizer` class:</b>
- `localizer.poses` : list of `geo_utils.Pose` objects
- `localizer.motion_update(pose, d_forward, d_theta, in_place = True)` 
  - update single pose to reflect movement from noisy odometry values
  - Inputs:
      - `pose`:       `geo_utils.Pose` object
      - `d_forward`:  expected distance to move forward pose
      - `d_theta`:    expected angle to turn pose in radians
      - `in_place`:   boolean flag to update `pose` or return new `geo_utils.Pose` object
- `localizer.sensor_update(pose, observation, in_place = True)` 
  - update single pose to reflect noisy observation
  - Inputs:
      - `pose`:       `geo_utils.Pose` object
      - `observation`:  list of `semantic_maps.Detection` objects that robot generated
      - `in_place`:   boolean flag to update `pose` or return new `geo_utils.Pose` object
- `localizer.normalize_poses()` 
  - update probability for every pose in `localizer.poses` so these probabilities sum to 1
- `localizer.resample()` 
  - sample `localizer.num_particles` number of particles from `localizer.poses`

In [3]:
def update_particles(localizer, d_forward = 0, d_theta = 0, observation = None, resample = True):
    '''
    description: run one loop of the particle filter update
    inputs:
        localizer - Localizer object
        d_forward - distance moved forward
        d_theta - angle turned to side (in radians CCW)
        observation - list of Detection objects
        resample - boolean indicating if resampling should be performed
    '''
    # update the positions and probabilities of each particle
    if not (d_forward == 0 and d_theta == 0):
        new_poses = []
        for pose in localizer.poses:
            # this for loop generates multiple motion samples per particle
            for i in range(localizer.num_motion_samples):
                new_pose = localizer.motion_update(pose, d_forward, d_theta, in_place = False)
                if localizer.map.isOnMap(Point(new_pose.x, new_pose.y)):
                    new_poses += [new_pose]
        localizer.poses = new_poses

    if observation != None:
        for pose in localizer.poses:
            localizer.sensor_update(pose, observation)


    localizer.normalize_poses()
    if resample:
        localizer.resample()    
        localizer.normalize_poses()

In [4]:
test_particle_filter(update_particles)

Actual location  :  (  162.6 , 379.2 )
Expected location:  (  168.7 , 371.0 )
Distance error:  10.2


Tests passed!!

## Map-making <a id="map-making"/>

The Cognitive Robotics Grand Challenge: Orienteering environment is purportedly similar to the testing environment we've created - so you should get acquainted with it! 

The Map(s) we provide you with for testing come in JSON format - a serialization of qualitative information. We have equipped you with a viewer that can view a simple map. 

In [4]:
'''
obj dictionary must contain:
    'width' : numeric width of map
    'height': numeric height of map
    'robot_pose':
        'x': x position
        'y': y position
        'theta': angle in radians
    'landmarks': list of (string object type, [(x, y), ... (x, y)]) tuples going CCW
    'path': list of (x, y) locations 
'''

# ** IMPORTANT -- add your object types to landmark_colors  **
# **     set the colors for PyPlot,  e.g. 'tree' : 'g'      **
# ** already included: 'house', 'tree', 'lake'

# visualization.landmark_colors['new_class'] = 'p'

my_map_file_name = 'my_map.json'
my_map = execute.BLANK_MAP

my_map['height'] = 400
my_map['width'] = 550
my_map['robot_pose']['x'] = 50
my_map['robot_pose']['y'] = 50
my_map['robot_pose']['theta'] = math.pi / 4
my_map['landmarks'] = [
    ('house', [(100, 100), (250, 100), (250, 200), (100, 200)]),
    ('house', [(350, 200), (425, 200), (425, 300), (350, 300)]),
    ('tree' , [(200, 300), (250, 300), (250, 350), (200, 350)])
    ]
my_map['path'] = [(50, 50), (50, 250), (300, 250), (300, 350), (500, 350), (500, 50)]

In [5]:
# save your map to a file
execute.write_to_json(my_map_file_name, my_map)

In [6]:
test_my_map(my_map_file_name)

Tests passed!!

## Test your localization algorithm <a id="test-methods"/>


Now you can put it all together. Using the map file that you just made or one of ours, adjust some of the parameters of the model to see how the performance changes. Try to understand what causes the algorithm to diverge or converge. In particular, consider how your parameter settings can make the algorithm susceptible to: 
- low density of initial states
- symmetries in the map
- pruning options too quickly
- not repopulating high probability poses often enough

In [7]:
# fine tune your parameters
localization_args = dict()
localization_args['map_file_name'] = 'my_map.json' # map file to use

localization_args['num_particles'] = 1200  # number of particles to maintain
localization_args['sense_every'] = 4       # frequency with which to obtain a new observation
localization_args['resample_every'] = 6    # frequency with with to resample

localization_args['sigma_angle'] = math.radians(5) # std dev of noisy turn motion update
localization_args['sigma_forward'] = 3.            # std dev of noisy forward motion update

In [8]:
test_localization(update_function = update_particles, **localization_args)

Actual location  :  (  56.6 , 56.6 )
Expected location:  (  279.2 , 200.6 )
Distance error:  265.1


Tests passed!!

## What is semantic about all this? <a id="semantic-ness"/>

Here, we ask you to look closely at your work (and, through yours, ours).

Summarize how changing the parameters affected performance.

What aspects of your map or path made localization easy or hard? 

Do you think performance would have improved if you used laser scan matching instead of semantic detections? Why? 

Congratulations! You're done!